In [53]:
from numpy import argmax
from pickle import load
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
# from nltk.translate.bleu_score import corpus_bleu
import json
import random
import csv
from numpy import array
from pickle import load
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers.merge import add
from keras.callbacks import ModelCheckpoint
import numpy as np



import os.path as osp
import os
from pprint import pprint
from skimage.transform import rescale, resize
from skimage import data, color, io
import skimage
import PIL
import scipy
import json
import os.path
from os import path
import tensorflow as tf

from keras.applications.xception import preprocess_input
from keras.applications.xception import Xception
from keras.models import Model
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.applications.imagenet_utils import decode_predictions

import sys






In [54]:
# from google.colab import drive
# drive.mount('/content/drive')

In [55]:
# SEED = 10
# #IMAGE_EMBEDDING_DIR = '/content/drive/My Drive/Colab_Notebooks/DL_data/CNNFeatureVectors/'
# IMAGE_EMBEDDING_DIR = '/content/drive/My Drive/Colab_Notebooks/DL_data/one_sample_cnn/'
# NUM_IMAGE_EMBEDDING_CHUNKS = 1
# GLOVE_EMBEDDING_FILE_NAME = '/content/drive/My Drive/Colab_Notebooks/DL_data/glove.6B.300d.txt'
# MAX_SEQUENCE_LENGTH = 92
# WORD_EMBEDDING_DIM = 300
# CAPTION_FILE_NAME = '/content/drive/My Drive/Colab_Notebooks/DL_data/image_to_caption.csv'

SEED = 10
IMAGE_EMBEDDING_DIR = '/Users/vinutahegde/Documents/USC/SEM3/DL/project/ws/'
NUM_IMAGE_EMBEDDING_CHUNKS = 1
GLOVE_EMBEDDING_FILE_NAME = 'glove.6B.300d.txt'
MAX_SEQUENCE_LENGTH = 92
WORD_EMBEDDING_DIM = 300
CAPTION_FILE_NAME = '/Users/vinutahegde/Documents/USC/SEM3/DL/project/ws/image_to_caption.csv'

In [56]:
class text_encoder:
    def get_embedding_matrix(self,filename, WORD_EMBEDDING_DIM):
        embeddings_index = {}
        word_to_index = {}

        with open(filename) as f:
            for index, line in enumerate(f):
                values = line.split()
                word = values[0]
                coefs = np.asarray(values[1:], dtype='float32')
                embeddings_index[word] = coefs
                word_to_index[word] = index

          # Vocabulary
        vocabulary = embeddings_index.keys()
        embedding_matrix = np.zeros((len(vocabulary) + 1, WORD_EMBEDDING_DIM))

        for word, i in word_to_index.items():
            embedding_vector = embeddings_index[word]
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector

        return embedding_matrix, vocabulary, word_to_index
    
    def load_embeddings(self):
        embedding_matrix, vocabulary, word_to_index = self.get_embedding_matrix(GLOVE_EMBEDDING_FILE_NAME, WORD_EMBEDDING_DIM)      
        self.embedding_matrix = embedding_matrix
        self.vocabulary = vocabulary
        self.word_to_index = word_to_index
        
    def get_sentence_embedding(self,sentence):
        words = sentence.split()    
        sentence_embedding =  np.zeros(shape=(MAX_SEQUENCE_LENGTH,WORD_EMBEDDING_DIM))
        mask =  np.zeros(MAX_SEQUENCE_LENGTH)
        i=0;
        for w in words:
            mask[i] = 1
            index = self.word_to_index.get(w,-1)
            if(index != -1):
                sentence_embedding[i] = self.embedding_matrix[index]
            else:
                sentence_embedding[i] = np.zeros(WORD_EMBEDDING_DIM)
            i+=1       
        return  sentence_embedding, mask

In [57]:
def data_generator(descriptions, photos, tokenizer, max_length, vocab_size):
# loop for ever over images
    while 1:
        for key, desc_list in descriptions.items():
            # retrieve the photo feature
            photo = photos[key][0]
            in_img, in_seq, out_word = create_sequences(tokenizer, max_length, desc_list, photo, vocab_size)
            yield [[in_img, in_seq], out_word]

In [58]:
def create_sequences(tokenizer, max_length, descriptions, photos, vocab_size):
    X1, X2, y = list(), list(), list()
    # walk through each image identifier
    for key, desc_list in descriptions.items():
        # walk through each description for the image
        for desc in desc_list:
            # encode the sequence
            seq = tokenizer.texts_to_sequences([desc])[0]
            # split one sequence into multiple X,y pairs
            for i in range(1, len(seq)):
                # split into input and output pair
                in_seq, out_seq = seq[:i], seq[i]
                # pad input sequence
                in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                # encode output sequence
                out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                # store
                X1.append(photos[key])
                X2.append(in_seq)
                y.append(out_seq)
    return array(X1), array(X2), array(y)

In [59]:
# generate a description for an image
def generate_desc(model, tokenizer, photo, max_length):
    # seed the generation process
    in_text = 'startseq'
    # iterate over the whole length of the sequence
    for i in range(max_length):
        # integer encode input sequence
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        # pad input
        sequence = pad_sequences([sequence], maxlen=max_length)
        # predict next word
        yhat = model.predict([photo,sequence], verbose=0)
        # convert probability to integer
        print(yhat)
        yhat = argmax(yhat)
        # map integer to word
        word = word_for_id(yhat, tokenizer)
        # stop if we cannot map the word
        if word is None:
            break
        # append as input for generating the next word
        in_text += ' ' + word
        # stop if we predict the end of the sequence
        if word == 'endseq':
            break
    return in_text

In [60]:
def Merge(dict1, dict2): 
    res = {**dict1, **dict2} 
    return res 

In [61]:
def getImageEmbedding():
    image_embedding = {}
    for i in range (NUM_IMAGE_EMBEDDING_CHUNKS):
        file_name = IMAGE_EMBEDDING_DIR + 'group_'+str(i+1)+'.json'
        with open(file_name) as json_file:
            print(file_name)
            json_data = json.load(json_file)
            json_data = json.loads(json_data)
            image_embedding = Merge(image_embedding, json_data) 
            image_embedding = json_data 
        return image_embedding

In [62]:
def getCaptions(id_list):
    caption_dict = {}
    with open(CAPTION_FILE_NAME) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            if row[1] in id_list:
                caption_dict[row[1]] = ['startseq ' + row[2] + ' endseq']
    return caption_dict

In [63]:

# convert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
    all_desc = list()
    for key in descriptions.keys():
#         print(descriptions[key])
        [all_desc.append(d) for d in descriptions[key]]
#     print(all_desc)
    return all_desc

In [64]:
def create_tokenizer(descriptions):
    lines = to_lines(descriptions)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

In [65]:
def vocab_fun(captions):
    index_to_word = {}
    word_to_index = {}
    all_words = {}
    for cap in captions:
        for word in cap.split(' '):
            all_words[word] = 1
    all_vocab=[w for w in all_words]
    index = 1
    for word in all_vocab:
        word_to_index[word] = index
        index_to_word[index] = word
        index += 1
    return (all_vocab, word_to_index, index_to_word)


In [66]:
def define_model(vocab_size, max_length):
    # feature extractor model
    inputs1 = Input(shape=(2048,))
    fe1 = Dropout(0.5)(inputs1)
    fe2 = Dense(256, activation='relu')(fe1)
    # sequence model
    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size, WORD_EMBEDDING_DIM, mask_zero=True)(inputs2)
    se2 = Dropout(0.5)(se1)
    se3 = LSTM(256)(se2)
    # decoder model
    decoder1 = add([fe2, se3])
    decoder2 = Dense(256, activation='relu')(decoder1)
    outputs = Dense(vocab_size, activation='softmax')(decoder2)
    # tie it together [image, seq] [word]
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.layers[2].set_weights([embedding_matrix])
    model.layers[2].trainable = False
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    # summarize model
    print(model.summary())
#     plot_model(model, to_file='model.png', show_shapes=True)
    return model

In [67]:
def max_length(descriptions):
# #     print(descriptions)
#     lines = to_lines(descriptions)
# #     print(lines)
# #     return max(len(d.split()) for d in lines)
    return 94

In [68]:
image_embd =  getImageEmbedding()
image_ids = list(image_embd.keys())
text_ebmd_encoder = text_encoder()
text_ebmd_encoder.load_embeddings()
caption_map = getCaptions(image_ids)
# print(caption_map)
all_vocab, wordtoix, ixtoword=vocab_fun(getCaptions(image_ids))
tokenizer = create_tokenizer(caption_map)

vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
# determine the maximum sequence length
max_length = 94 #max_length(caption_map)
print(max_length)
X1train, X2train, ytrain = create_sequences(tokenizer, max_length, caption_map, image_embd, vocab_size)

X1test, X2test, ytest = create_sequences(tokenizer, max_length, caption_map, image_embd, vocab_size)



/Users/vinutahegde/Documents/USC/SEM3/DL/project/ws/group_1.json
Vocabulary Size: 4386
94


In [69]:


embeddings_index = {} # empty dictionary
f = open(GLOVE_EMBEDDING_FILE_NAME, encoding="utf-8")

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))
embedding_dim = 300

# Get 300-dim dense vector for each of the 10000 words in out vocabulary
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in wordtoix.items():
    #if i < max_words:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in the embedding index will be all zeros
        embedding_matrix[i] = embedding_vector

Found 400000 word vectors.


In [70]:
# model = define_model(vocab_size, max_length)
# filepath = 'model-ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5'
# checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
# model.fit([X1train, X2train], ytrain, epochs=2, verbose=2, callbacks=[checkpoint])#, validation_data=([X1test, X2test], ytest))
#model.save('/content/drive/My Drive/Colab_Notebooks/DL_data/saved_model_1.h5')
model = define_model(vocab_size, max_length)
filepath = 'model-ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
model.fit([X1train, X2train], ytrain, epochs=2, verbose=2, callbacks=[checkpoint])

Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_20 (InputLayer)           (None, 94)           0                                            
__________________________________________________________________________________________________
input_19 (InputLayer)           (None, 2048)         0                                            
__________________________________________________________________________________________________
embedding_8 (Embedding)         (None, 94, 300)      1315800     input_20[0][0]                   
__________________________________________________________________________________________________
dropout_15 (Dropout)            (None, 2048)         0           input_19[0][0]                   
___________________________________________________________________________________________

KeyError: 'val_loss'

In [ ]:
#save model
model.save('model-ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5')


In [ ]:
def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None
def load_image(image_path,target_size):
    img = skimage.io.imread(image_path)
    image_resized = skimage.transform.resize(img, target_size, anti_aliasing=True)
    return image_resized

def load_cnn_model():
    model = Xception()
    model.layers.pop()
    model = Model(inputs=model.inputs, outputs=model.layers[-1].output)
    return model

def extract_features_from_images(image_path):
    model = load_cnn_model()
    if path.exists(image_path):
        print(image_path)
        image = load_image(image_path, target_size=(299, 299))
        if image.shape == (299, 299, 3):
            image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
            image = preprocess_input(image)
            feature = model.predict(image, verbose=0)
            print(feature)
            return feature

In [ ]:
image_path='/Users/vinutahegde/Documents/USC/SEM3/DL/project/ws/images/train/images/train/124919.jpg'
feature=extract_features_from_images(image_path)
print(generate_desc(model, tokenizer, feature, max_length))

In [ ]:
image_path='/Users/vinutahegde/Documents/USC/SEM3/DL/project/ws/images/train/images/train/124871.jpg'
feature=extract_features_from_images(image_path)
test_features = np.ones(shape=(1, 2048))
print(generate_desc(model, tokenizer, test_features, max_length))